# Pre Lab 11 : Markov Chains

## Objectives

Study one use of random numbers: random processes in the form of Markov Chains.


## Course Evaluations

Complete the course evaluations at this link: [Course Evaluations](https://webapps.case.edu/courseevals/). (I am not sure when they will be made available, but it should be some time before Thursday.)

## Initialization

As always you should add initialization to the top of your notebook.

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
#Globally fix plot styling
import matplotlib as mpl
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

rng = np.random.default_rng()

## Markov Chains

A Markov chain (strictly speaking the discrete time Markov chain) descibes a random process in which a system evolves by moving among a set of allowed states with given probabilities.  It has the added property that it is "memoryless"; the probability of moving from one state to another does not depend on the previous states visited.

### Where they appear

Many physical processes can be described by Markov chains.  Applications are extremely far-reaching, spanning many disciplines and areas of research.  Examples of applications of Markov chains include search engine rankings, financial market behavior, and even simulations of brain activity. See the [Wikipedia page](http://en.wikipedia.org/wiki/Markov_chain#Applications) for a list of a few more examples. Numerous extensions to the simple model of a Markov chain presented here also exist, some of which are discussed on the Wikipedia page. One extension you may have heard of are [Markov chain Monte Carlo](http://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo) simulations, which combines Monte Carlo techniques mentioned in class and the lab with Markov chain techniques.

Due to the probabilistic nature of Markov chains, they frequently appear in physics in statistical and thermodynamic systems. Here we will look at a couple of examples of Markov chains in order to predict the behavior of a simple system.

### Defining the model

Markov chains model systems as a set of discrete states with some probability of transitioning from one state to another (including the possibility of remaining in the original state). At any given time there will be some probability of being in any one of the allowed states.  Let this set of probabilities be represented by a vector $\vec{q}$. As a simple example, let us consider a 3-state system, which we can write as $\vec{q} = (q_1, q_2, q_3)$. Since this is a set of probabilities and we must be in one of the states, the sum of the components of this vector must be unity. When in a given state there will be some probability of transitioning to another state: let us call $P_{i \rightarrow j}$ the probability of transitioning from state $i$ to state $j$. We can represent the transition probabilities for a 3 state system as
![Markov Chain States](https://github.com/cwru-phys-250/p250-spring-2022/raw/main/figures/states.png)

As an example of these ideas suppose we initially begin in state $1$ with probability $q_1^i = 1$.  This is represented by the initial state vector $\vec{q}^i = (1,0,0)$. Then the probability of transitioning to state $3$ is given by $P_{1 \rightarrow 3}$ and similarly for the other states. So after one time step the state vector will be $\vec{q}^f = (P_{1 \rightarrow 1},P_{1 \rightarrow 2},P_{2 \rightarrow 3})$. We can perform a similar calculation for the other initial states (starting in state 2 or state 3 with probability one).  The general evolution of the state vector in a Markov chain can be represented by a set of linear equations, which we can write in matrix form as
$$
\begin{pmatrix}
q_{1}^{f}\\
q_{2}^{f}\\
q_{3}^{f}
\end{pmatrix}
=
\begin{pmatrix}
P_{1\rightarrow1} & P_{2\rightarrow1} & P_{3\rightarrow1}\\
P_{1\rightarrow2} & P_{2\rightarrow2} & P_{3\rightarrow2}\\
P_{1\rightarrow3} & P_{2\rightarrow3} & P_{3\rightarrow3}
\end{pmatrix} 
\begin{pmatrix}
q_{1}\\
q_{2}\\
q_{3}
\end{pmatrix}.
$$

Analyzing the transition matrix, $\mathsf{P}$, we can find a number of special properties.  One of the intuitive properties is that each of its columns must sum to unity; this being nothing more than conservation of probability. A less obvious property is that the eigenvalues of the system will all be unity or less.  Further, if all the states of the system are accessible after some number of state changes for all possible initial states, then there will be one eigenvalue exactly equal to one. This eigenvalue and its associated eigenvector will describe the behavior of the system after a large number of state changes.

To better understand this we again let the system start in the initial state $\vec{q}^i$.  After one state change (one step in the Markov chain) the probability of being in any state will be $\mathsf{P} \vec{q}^i$. After another state change it will be $\mathsf{P}^2 \vec{q}^i$, and after $n$ changes it will be $\mathsf{P}^n \vec{q}^i$. Recall from homework 9 that you can raise a matrix to an arbitrary power by diagonalizing it, and raising the resulting matrix to a power. This results in the relationship 

$$ \mathsf{P}^n = \mathsf{V} \mathsf{D}^n \mathsf{V}^{-1} $$

where $\mathsf{V}$ is a matrix whose columns are the eigenvectors of $\mathsf{P}$ and $\mathsf{D}$ is a matrix containing the corresponding eigenvalues along the diagonal.

It is interesting to consider what happens in the $n \rightarrow \infty$ limit. As we saw in the homework problem mentioned above, since one eigenvalue is unity and the rest smaller than one, taking the limit as $n \rightarrow \infty$ will result in all eigenvalues except the unitary one approaching zero.  The eigenvector corresponding to this eigenvalue is somewhat special: if the probability of being in a given state is described by this eigenvector, after subsequent steps the probability of being in a state remains the same; while the probability of being in a state described by other eigenvectors will diminish. Thus after a long time, the probability of being in a given state will be described by the eigenvector corresponding to the unit eigenvalue. This vector is sometimes referred to as the **stationary state of the system**.  After a long time, or equivalently many steps, the system will settle into its stationary state.

## Cleveland Weather

As a simple model of spring weather in Cleveland let us consider a 3-state system with the states representing a "sunny" day, a "rainy" day, and a "cloudy" day. A more realistic model may include knowledge of weather at other locations, temperatures humidity; or a full-blown climate simulation. However for a simple example we will ignore the details. Let the probabilities of transitioning from one type of weather to another be given as in the following diagram:
![Weather States](https://github.com/cwru-phys-250/p250-spring-2022/raw/main/figures/weather_states.png)

Write the transition matrix $\mathsf{P}$ that describes this diagram for the state vector
$$\vec{q} = \begin{pmatrix} q_☼ \\ q_☂ \\ q_☁ \end{pmatrix}.$$
Verify that the columns of this matrix add up to one.

In [32]:
P=np.array([
    [0.05,0.0,0.2],
    [0.25,0.3,0.35],
    [0.7,0.7,0.45]

])
np.sum(P,axis=0)

array([1., 1., 1.])

Perform an eigenvalue decomposition of this matrix.  You should find all the eigenvalues are real (even though $\mathsf{P}$ is not symmetric) and one of them is unity.  Print the eigenvector corresponding to the unit eigenvalue.

In [33]:
(lam, B) = la.eig(P)
print("Eigenvalues:", lam)
print("Matrix of eigenvectors:\n", B[:,0])

Eigenvalues: [ 1.  +0.j  0.05+0.j -0.25+0.j]
Matrix of eigenvectors:
 [-0.17952684 -0.49049296 -0.85275247]


This is good, but this vector looks wrong.  It is suppose to represent probabilities and the sum must be one.  It may also be the case that all the values are negative, so how could they be probabilities!  The case of negative values can easily be fixed by multiplying the vector by negative one.  (Recall why we are allowed to do this!)  The other issue is due to how eigenvectors are normalized.  (Again recall how this is done!)  Since we have a physical meaning for the vector and its normalization we will need to renormalize it.  (For different reasons and in a different manner we needed to renormalize the eigenvectors when we solved the Schrödinger equation in Lab 9.)

Renormalize the eigenvector from the previous part so that its sum is one.  Print the renormalized vector.  (All the probabilities better be positive now!)

In [34]:
B=-1*B
e1=B[:,0]
e1_norm=e1/np.sum(e1)
print(e1_norm)


[0.11789474 0.32210526 0.56      ]


We can use our simple model to forecast the weather.  Given an initial state representing the weather today we can predict the probabilities of it being sunny, rainy, or cloudy after 1 day, 2 days, or 1 week.  To begin pick an initial state vector representing the weather today
$$ \vec{q}^i = \left( \begin{array}{c} q_☼ \\ q_☂ \\ q_☁ \end{array} \right). $$
Pick one that is somewhat representative of the weather when you are working on the prelab!

In order to calculate the weather after one, two, and seven days you will need to calculate $\mathsf{P} \vec{q}^i$, $\mathsf{P}^2 \vec{q}^i$, and $\mathsf{P}^7 \vec{q}^i$. There are multiple ways to raise a matrix to a power. One way is to use the eigenvalue decomposition from above (and as discussed in homework 9). As an alternative, NumPy provides a function for doing this: `np.linalg.matrix_power`.  Either of these methods is better than repeatedly multiplying the matrix with itself.

Print the probabilities for the state of the weather after 1, 2, and 7 days.  Note that after 7 days the result should look very much like the stationary state we found above.  For fun you may want to compare your predictions to those from a professional weather service and/or keep track of you predictions and see how they turn out.

In [35]:
q_1=np.array([[1],[0],[0]])
P_2=np.linalg.matrix_power(P,2)
P_7=np.linalg.matrix_power(P,7)
print(f"Weather after one day \n {P@q_1} \nWeather after two day \n {P_2@q_1} \n Weather after seven day \n {P_7@q_1} \n")

Weather after one day 
 [[0.05]
 [0.25]
 [0.7 ]] 
Weather after two day 
 [[0.1425]
 [0.3325]
 [0.525 ]] 
 Weather after seven day 
 [[0.11787195]
 [0.32209387]
 [0.56003418]] 



## Drunkard's Walk

A random walk is another simple example of a Markov chain.  Here we will work through a variant of the random walk where we add the special property that it is possible to get stuck in certain states. This is called an "**absorbing chain**" with the transition probabilities given in the diagram below.
![Example of an absorbing chain](https://github.com/cwru-phys-250/p250-spring-2021/raw/main/figures/absorbing_chain.png)

We can consider this to represent the path taken by an inebriated individual as he attempts to locate either his home or another bar. In state 1 the drunkard reaches home and will remain there.  In state 5 he reaches another bar and again remains there.  In between the two destinations he will stumble in either direction with probability $1/2$ (this equal probability step is the usual random walk behavior).

Systems like this are special in that more than one eigenvalue is unity so there are multiple stationary states. (The assertion made above that all states should be accessible has been broken.)

Construct the transition matrix, $\mathsf{P}$, for this system.   Print the vectors representing the stationary states of the system. 

In [39]:
P=np.array([
    [1.,0.5,0.,0.,0.],
    [0.,0.,0.5,0.,0.],
    [0.,0.5,0.,0.5,0],
    [0,0,0.5,0,0.],
    [0,0,0,0.5,1]
])
(lam, B)=la.eig(P)
print(f"the stationary states vectors are:\n{B[:,0:2]}")

the startionary states vectors are:
[[1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]]


The form of the stationary states make sense.  Describe in words what they represent.  Are all states accessible from all other states?

These vectors correspond to the drunkard getting stuck a  node 1 or 5. Not all states are accessible, since node 1 and node 5 are looped to themselves.

There are many other interesting questions that could be asked.  For example, suppose the drunkard starts in state labeled 2 in the figure, what is the probability he will end up at home?  Similarly, what are the probabilities when starting the in the states labeled 3 or 4?  For each of these initial states we could also determine the average number of steps taken by the drunkard.  Numerically these can be easily modeled using Monte Carlo techniques to simulate a drunkard walking from each initial state.  We will not do so here, but feel free to 
explore it on your own.

## Turning in the PreLab

All prelabs will be handled as was done for PreLab01.  See that file for details.  It will be assumed from now on that you have read and understood the procedure and what it means when you submit a prelab.